<a href="https://colab.research.google.com/github/Ak-Gautam/efficient_llm_fine_tunes/blob/main/techniques/dataset_chop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Dataset shaving

A trimmed down but balanced version of the awesome [OpenHermes](https://huggingface.co/datasets/teknium/OpenHermes-2.5) dataset for resource restricted usecases.

In [ ]:
!pip install huggingface_hub -q
!pip install datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [ ]:
from datasets import load_dataset

dataset = load_dataset("teknium/OpenHermes-2.5", split='train')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1001551 [00:00<?, ? examples/s]

In [ ]:
import pandas as pd
import time
from sklearn.model_selection import train_test_split
start_time = time.time()
# Convert the dataset to a pandas DataFrame
df = pd.DataFrame(dataset)

# Now, sampled_df is a DataFrame with 110k rows, evenly distributed among all the values of source
end_time = time.time()
print(f"Elapsed time: {end_time - start_time:.2f} seconds")

Elapsed time: 252.57 seconds


#### Version 2

In [ ]:
from collections import Counter

# Group the DataFrame by 'source' and calculate the count of each source
source_counts = df['source'].value_counts()

# Calculate the desired number of rows for each source
target_rows_per_source = 110_000 // len(source_counts) + 4000

sampled_rows = []
for source, count in source_counts.items():
    source_df = df[df['source'] == source]
    sampled_source_df = source_df.sample(min(count, target_rows_per_source), replace=False)
    sampled_rows.append(sampled_source_df)

# Concatenate the sampled rows into a single DataFrame
sampled_df = pd.concat(sampled_rows, ignore_index=True)

In [ ]:
source_counts

source
glaive-code-assist       182240
CamelAI                   78390
metamath                  56448
EvolInstruct_70k          51948
cot_alpaca_gpt4           42026
airoboros2.2              35380
platypus                  22280
GPT-4 Comparison Data     14928
UnnaturalInstructions      8610
CogStackMed                4443
LMSys Chatbot Arena        3136
caseus_custom              2688
lmsys1m                    1631
Econ_domain_expert          660
Name: count, dtype: int64

In [ ]:
sampled_df['source'].value_counts()

source
glaive-code-assist       11857
CamelAI                  11857
metamath                 11857
EvolInstruct_70k         11857
cot_alpaca_gpt4          11857
airoboros2.2             11857
platypus                 11857
GPT-4 Comparison Data    11857
UnnaturalInstructions     8610
CogStackMed               4443
LMSys Chatbot Arena       3136
caseus_custom             2688
lmsys1m                   1631
Econ_domain_expert         660
Name: count, dtype: int64

In [ ]:
sampled_df.shape

(116024, 16)

In [ ]:
# Convert the DataFrame back to a Dataset
sampled_dataset = dataset.from_pandas(sampled_df[['source', 'conversations']])

In [ ]:
# Shuffle the dataset with a seed for reproducibility
seed = 42
shuffled_dataset = sampled_dataset.shuffle(seed=seed)

In [ ]:
print(shuffled_dataset)

Dataset({
    features: ['source', 'conversations'],
    num_rows: 116024
})


#### Upload dataset

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
sampled_dataset.push_to_hub("glouriousgautam/openhermes116k_balanced")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/117 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/glouriousgautam/openhermes116k_balanced/commit/38d1b76d24bec0819d7f474c973f04b42556e864', commit_message='Upload dataset', commit_description='', oid='38d1b76d24bec0819d7f474c973f04b42556e864', pr_url=None, pr_revision=None, pr_num=None)